## CPU Bound workloads

This notebook demonstrates using Dask-ML to help with CPU-bound workloads. We'll see an example of training a large ensemble model (`RandomForsetClassifier`) and a large grid search.

In [2]:
import dask.dataframe as dd
from dask.distributed import Client
from dask_kubernetes import KubeCluster

In [ ]:
cluster = KubeCluster(n_workers=100)
cluster

In [ ]:
client = Client(cluster)

In [3]:
df = dd.read_parquet("gs://dask-nyc-taxi/yellowtrip.parquet",
                     engine="fastparquet",
                     storage_options={"token": "anon"})
df

,vendor_id,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,tolls_amount,total_amount
npartitions=907,,,,,,,,,,,,,,
2014-01-01 00:00:00,category[unknown],datetime64[ns],int64,float64,float64,float64,category[unknown],float64,float64,category[unknown],float64,float64,float64,float64
2014-01-02 00:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 00:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 23:59:59,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Read in a small subset of the data. This fits in memory (on our client machine, and on each of the workers).

In [4]:
sdf = df.partitions[0].compute()
sdf = sdf.head(200_000)

sdf.head()

,vendor_id,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,tolls_amount,total_amount
pickup_datetime,,,,,,,,,,,,,,
2014-01-01,VTS,2014-01-01 00:50:00,1,41.62,-73.871285,40.773962,nassau,-73.618775,41.200682,CSH,179.0,0.0,5.33,185.33
2014-01-01,VTS,2014-01-01 00:18:00,1,5.58,-73.961757,40.759820,standard,-73.978477,40.743305,CSH,17.0,0.0,0.00,18.00
2014-01-01,VTS,2014-01-01 00:28:00,3,6.20,-73.956543,40.778122,standard,-74.000382,40.718385,CRD,23.0,4.7,0.00,28.70
2014-01-01,VTS,2014-01-01 00:22:00,1,7.93,-73.979027,40.749187,standard,-73.899360,40.828842,CSH,25.0,0.0,0.00,26.00
2014-01-01,VTS,2014-01-01 00:03:00,6,1.05,-73.971300,40.792705,standard,-73.959910,40.800747,CSH,5.0,0.0,0.00,6.00


## Load the Data

In [13]:
import pandas as pd

In [16]:
rides = pd.read_parquet("data/taxi-small.parquet", engine="fastparquet")
rides.head()

,vendor_id,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,tolls_amount,total_amount
pickup_datetime,,,,,,,,,,,,,,
2014-01-01,VTS,2014-01-01 00:50:00,1,41.62,-73.871285,40.773962,nassau,-73.618775,41.200682,CSH,179.0,0.0,5.33,185.33
2014-01-01,VTS,2014-01-01 00:18:00,1,5.58,-73.961757,40.759820,standard,-73.978477,40.743305,CSH,17.0,0.0,0.00,18.00
2014-01-01,VTS,2014-01-01 00:28:00,3,6.20,-73.956543,40.778122,standard,-74.000382,40.718385,CRD,23.0,4.7,0.00,28.70
2014-01-01,VTS,2014-01-01 00:22:00,1,7.93,-73.979027,40.749187,standard,-73.899360,40.828842,CSH,25.0,0.0,0.00,26.00
2014-01-01,VTS,2014-01-01 00:03:00,6,1.05,-73.971300,40.792705,standard,-73.959910,40.800747,CSH,5.0,0.0,0.00,6.00


In [18]:
features = [
    'passenger_count', 'trip_distance',
    'pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude',
    'fare_amount', 'tolls_amount', 'total_amount',
]

y = (rides['tip_amount'] > 0).astype(int)
X = rides[features]

## Fitting an Ensemble with Scikit-Learn

We'll make a small pipeline to predict whether the tip was positive. The pipeline will

1. Scale the data
2. Fit a random forest

using just scikit-learn. This will be done in parallel **on a single machine** (8 cores in this case).

In [23]:
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.ensemble

In [28]:
pipeline = sklearn.pipeline.make_pipeline(
    sklearn.preprocessing.StandardScaler(),
    sklearn.ensemble.RandomForestClassifier(
        n_estimators=100,
        n_jobs=-1,
    )
)

In [27]:
%time _ = pipeline.fit(X, y)

CPU times: user 1min 10s, sys: 538 ms, total: 1min 10s
Wall time: 10.5 s


## Fitting an Ensemble on a Dask Cluster

Let's suppose we want to scale our model by fitting many more trees. Let's use 1,000 instead of 100. This would take about 10x longer on our single machine. We'll improve on this by using the Dask cluster to do the training, rather than a single machine.

In [29]:
pipeline = sklearn.pipeline.make_pipeline(
    sklearn.preprocessing.StandardScaler(),
    sklearn.ensemble.RandomForestClassifier(
        n_estimators=1000,  # 100 -> 1,000
        n_jobs=-1,
    )
)

This new `pipeline` *could* be fit on a single machine with

```python
pipeline.fit(X, y)
```

But this would take to long. To use a cluster, ensure that the fit call happens in a `joblib.parallel_backend` context manager.

In [14]:
%%time
import joblib

with joblib.parallel_backend("dask"):
    pipe.fit(X, y)

CPU times: user 22.6 s, sys: 3.3 s, total: 25.9 s
Wall time: 36 s


A few things to note

* The Dask dashboard shows that the workers is active during the training.
* We never see all the tasks (1,000 in this case) waiting at once. Joblib does some dispatching to not submit all the tasks at once.
* We don't achieve perfect scaling. We have 12.5 as many cores, but don't complete training 12.5 times faster than we would on a single machine.
  * The model and data are on different machines. It must be serialized, shipped over the network, and deserialized before training can really begin (distributed computing is hard).
  * Joblib and Dask each have some overhead (which could surely be improved)

In [15]:
cluster.scale(250)
cluster

## Distributed Grid Search

Dask-ML provides drop-in replacements for GridSearchCV and RandomizedSearchCV. While `joblib.parallel_backend` *could* be used, we recommend using the ones in `dask_ml.model_selection`. Dask deconstructs the grid search into its component tasks. This provides

1. Much nicer diagnostics with the Dask dashboard.
2. Improved performance when using pipelines. Dask knows how to hash inputs to avoid redundant computation.

In [30]:
import sklearn.experimental.enable_hist_gradient_boosting
import dask_ml.model_selection

In [31]:
pipe = sklearn.pipeline.make_pipeline(
    sklearn.preprocessing.StandardScaler(),
    sklearn.ensemble.HistGradientBoostingClassifier(n_iter_no_change=3)
)

# param_grid = dict(
#     histgradientboostingclassifier__learning_rate=[0.009, 0.01, 0.011],
#     histgradientboostingclassifier__max_iter=[90, 100, 110],
#     histgradientboostingclassifier__max_leaf_nodes=[25, 30, 35],
#     histgradientboostingclassifier__max_depth=[3, 4, 5],  # these are all too small
# )
# search = dask_ml.model_selection.GridSearchCV(pipe, param_grid, cv=3)
# search

In [42]:
search.best_estimator_[1].get_params()

{'l2_regularization': 0.0,
 'learning_rate': 0.1,
 'loss': 'auto',
 'max_bins': 255,
 'max_depth': None,
 'max_iter': 100,
 'max_leaf_nodes': 31,
 'min_samples_leaf': 20,
 'n_iter_no_change': 3,
 'random_state': None,
 'scoring': None,
 'tol': 1e-07,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [49]:
%%time

param_grid = dict(
#     histgradientboostingclassifier__learning_rate=[0.009, 0.01, 0.011],
#     histgradientboostingclassifier__max_iter=[90, 100, 110],
#     histgradientboostingclassifier__max_leaf_nodes=[25, 30, 35],
#     histgradientboostingclassifier__max_depth=[3, 4, 5],  # these are all too small
    histgradientboostingclassifier__max_leaf_nodes=[25, 30, 35],
    histgradientboostingclassifier__min_samples_leaf=[20, 25, 30]
)

search = dask_ml.model_selection.GridSearchCV(pipe, param_grid, cv=3)

search.fit(X, y)

CPU times: user 2min 40s, sys: 1min 37s, total: 4min 18s
Wall time: 49.1 s


GridSearchCV(cache_cv=True, cv=3, error_score='raise',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('histgradientboostingclassifier',
                                        HistGradientBoostingClassifier(l2_regularization=0.0,
                                                                       learning_rate=0.1,
                                                                       loss='auto',
                                                                       max_bins=255,
                                                                       max_depth=None,
                                                                       max_iter=100,
                                  

In [19]:
import numpy as np

search.cv * np.prod([len(x) for x in param_grid.values()])

243

For brevity, we'll also scale the dataset down to a 1/4th its original size.

In [20]:
X = X[::4]
y = y[::4]

Now we fit the model, using the Dask cluster to perform the computation.

In [21]:
%time search.fit(X, y);

CPU times: user 1min 1s, sys: 3.87 s, total: 1min 5s
Wall time: 1min 56s


GridSearchCV(cache_cv=True, cv=3, error_score='raise',
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categorical_features=None,
                                                                                       categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float6

In [23]:
search.best_score_

0.57558

In [24]:
cv_results = pd.DataFrame(search.cv_results_)
cv_results

,params,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,param_histgradientboostingclassifier__learning_rate,param_histgradientboostingclassifier__max_depth,param_histgradientboostingclassifier__max_iter,param_histgradientboostingclassifier__max_leaf_nodes
0,{'histgradientboostingclassifier__learning_rat...,24.202364,7.411239,0.641995,0.047167,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.009,3,90,25
1,{'histgradientboostingclassifier__learning_rat...,28.786151,0.587387,0.587808,0.057334,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.009,3,90,30
2,{'histgradientboostingclassifier__learning_rat...,29.051803,1.044604,0.612409,0.097590,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.009,3,90,35
3,{'histgradientboostingclassifier__learning_rat...,27.637108,1.379088,0.472969,0.243787,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.009,3,100,25
4,{'histgradientboostingclassifier__learning_rat...,28.544266,1.454357,0.527614,0.245079,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.009,3,100,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,{'histgradientboostingclassifier__learning_rat...,66.898165,2.780781,0.372961,0.228566,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.011,5,100,30
77,{'histgradientboostingclassifier__learning_rat...,69.970422,0.189035,0.507991,0.273945,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.011,5,100,35
78,{'histgradientboostingclassifier__learning_rat...,66.410965,1.519031,0.636783,0.103248,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.011,5,110,25
79,{'histgradientboostingclassifier__learning_rat...,69.032318,2.494337,0.568528,0.138885,0.575568,0.575568,0.575603,0.57558,0.000016,1,0.011,5,110,30


## Transition: Local training, Distributed Prediction

Sometimes, you need to predict / score for a much larger dataset than training.
There may be no benefit to training on more samples, or labels may be hard to obtain.

If the training can be done in-memory on a single machine, but the population for prediction is larger-than-memory, `dask_ml.wrappers.ParallelPostFit` will help.

In [25]:
from dask_ml.wrappers import ParallelPostFit

In [28]:
sklearn_estimator = sklearn.pipeline.make_pipeline(
    preprocess,
    sklearn.ensemble.HistGradientBoostingClassifier(n_iter_no_change=5)
)

pipe2 = ParallelPostFit(
    sklearn_estimator,
    scoring='accuracy'
)
pipe2

ParallelPostFit(estimator=Pipeline(memory=None,
                                   steps=[('columntransformer',
                                           ColumnTransformer(n_jobs=None,
                                                             remainder='passthrough',
                                                             sparse_threshold=0.3,
                                                             transformer_weights=None,
                                                             transformers=[('onehotencoder',
                                                                            OneHotEncoder(categorical_features=None,
                                                                                          categories='auto',
                                                                                          drop=None,
                                                                                          dtype=<class 'numpy.float64'>,
                 

In [31]:
pipe2.fit(X, y);

Now this estimator can be used for prediction, scoring, and transformation on
larger than memory datasets. It can be serialized and loaded just like any
other scikit-learn estimator.

In [34]:
pipe2.score(X, y)

0.98038

In [36]:
# fixup categorical dtypes
head = df.head()
dtypes = head.select_dtypes(include='category').dtypes

for col, dtype in dtypes.items():
    df[col] = df[col].astype(dtype)

In [37]:
df = df.persist()

In [38]:
y_big = (df['tip_amount'] > 0).astype(int)
X_big = df.drop("tip_amount", axis="columns")

In [39]:
X_big

,vendor_id,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tolls_amount,total_amount
npartitions=907,,,,,,,,,,,,,
2014-01-01 00:00:00,category[known],datetime64[ns],int64,float64,float64,float64,category[known],float64,float64,category[known],float64,float64,float64
2014-01-02 00:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 00:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 23:59:59,...,...,...,...,...,...,...,...,...,...,...,...,...


In [41]:
predictions = pipe2.predict(X_big)
predictions

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [42]:
predictions = predictions.persist()

In [43]:
predictions.compute_chunk_sizes()
predictions

,Array,Chunk
Bytes,2.88 GB,4.60 MB
Shape,"(359892261,)","(574524,)"
Count,907 Tasks,907 Chunks
Type,int64,numpy.ndarray


In [44]:
predictions[:5].compute()

array([0, 0, 1, 0, 0])

In [50]:
y_big.head()

pickup_datetime
2014-01-01    0
2014-01-01    0
2014-01-01    1
2014-01-01    0
2014-01-01    0
Name: tip_amount, dtype: int64

In [45]:
%time pipe2.score(X_big, y_big)

CPU times: user 21.8 s, sys: 876 ms, total: 22.6 s
Wall time: 25.7 s


0.9816349899227202

## Plot a learning Curve

In [52]:
cluster.scale(0)